## NORMAL

In [96]:
ISU = 501489
N = ISU % 20

In [ ]:
import random
from itertools import product, permutations, combinations
from typing import List, Set, Dict, Tuple
from math import gcd


class Permutation:
    """Класс для работы с перестановками."""
    def __init__(self, mapping: List[int]):
        self.n = len(mapping)
        self.mapping = [0] + mapping
        self._cycles = None
        self._order = None
    
    def __mul__(self, other):
        if self.n != other.n:
            raise ValueError("Перестановки должны быть одинакового размера")

        result = [0] * (self.n + 1)
        for i in range(1, self.n + 1):
            result[i] = self.mapping[other.mapping[i]]
        return Permutation(result[1:])
    
    def __pow__(self, power):
        if power == 0:
            return Permutation(list(range(1, self.n + 1)))
        if power < 0:
            return self.inverse() ** (-power)

        result = Permutation(list(range(1, self.n + 1)))
        base = self
        while power > 0:
            if power % 2 == 1:
                result = result * base
            base = base * base
            power //= 2
        return result
    
    def inverse(self):
        inv = [0] * (self.n + 1)
        for i in range(1, self.n + 1):
            inv[self.mapping[i]] = i
        return Permutation(inv[1:])
    
    def cycles(self):
        if self._cycles is not None:
            return self._cycles

        visited = [False] * (self.n + 1)
        cycles = []

        for i in range(1, self.n + 1):
            if not visited[i]:
                cycle = []
                j = i

                while not visited[j]:
                    visited[j] = True
                    cycle.append(j)
                    j = self.mapping[j]

                if len(cycle) > 1:
                    cycles.append(tuple(cycle))

        self._cycles = cycles
        return cycles
    
    def order(self):
        def lcm(a, b):
            return a * b // gcd(a, b)

        if self._order is not None:
            return self._order

        cycles = self.cycles()
        if not cycles:
            self._order = 1
            return 1

        lengths = [len(cycle) for cycle in cycles]
        result = lengths[0]
        for length in lengths[1:]:
            result = lcm(result, length)

        self._order = result
        return result
    
    def __eq__(self, other):
        return self.mapping == other.mapping
    
    def __hash__(self):
        return hash(tuple(self.mapping))
    
    def __str__(self):
        cycles = self.cycles()
        if not cycles:
            return "()"
        return " ".join(f"({' '.join(map(str, cycle))})" for cycle in cycles)


def generate_symmetric_group(n: int) -> List[Permutation]:
    """Генерирует все элементы симметрической группы S_n."""
    elements = []
    for perm_tuple in permutations(range(1, n + 1)):
        elements.append(Permutation(list(perm_tuple)))
    return elements


def subgroup_generated_by(generators: List[Permutation]) -> Set[Permutation]:
    """Находит подгруппу, порожденную данными элементами."""
    if not generators:
        return set()
    subgroup = set()
    queue = [Permutation(list(range(1, generators[0].n + 1)))]
    while queue:
        current = queue.pop(0)
        if current in subgroup:
            continue
        subgroup.add(current)
        for gen in generators:
            for new_elem in [current * gen, gen * current]:
                if new_elem not in subgroup:
                    queue.append(new_elem)
    return subgroup


def find_all_subgroups(S_n: List[Permutation]) -> List[Set[Permutation]]:
    """Находит все подгруппы симметрической группы."""
    n = S_n[0].n

    identity = S_n[0]
    all_subgroups = [set([identity])]
    seen_subgroups = {frozenset([identity])}

    queue = [set([identity])]
    
    while queue:
        current_subgroup = queue.pop(0)
        
        for g in S_n:
            if g in current_subgroup:
                continue
            
            generators = list(current_subgroup) + [g]
            new_subgroup = subgroup_generated_by(generators)
            new_subgroup_frozen = frozenset(new_subgroup)
            
            if new_subgroup_frozen not in seen_subgroups:
                seen_subgroups.add(new_subgroup_frozen)
                all_subgroups.append(new_subgroup)
                
                if len(new_subgroup) < len(S_n):
                    queue.append(new_subgroup)
    
    full_group_frozen = frozenset(S_n)
    if full_group_frozen not in seen_subgroups:
        all_subgroups.append(set(S_n))
    
    return all_subgroups


def left_cosets(G: List[Permutation], H: Set[Permutation]) -> List[Set[Permutation]]:
    """Находит левые смежные классы."""
    cosets = []
    used_elements = set()
    
    for g in G:
        if g in used_elements:
            continue
        
        # Вычисляем смежный класс g*H
        coset = set()
        for h in H:
            elem = g * h
            coset.add(elem)
            used_elements.add(elem)
        
        cosets.append(coset)
    
    return cosets


def right_cosets(G: List[Permutation], H: Set[Permutation]) -> List[Set[Permutation]]:
    """Находит правые смежные классы."""
    cosets = []
    used_elements = set()
    
    for g in G:
        if g in used_elements:
            continue

        coset = set()
        for h in H:
            elem = h * g
            coset.add(elem)
            used_elements.add(elem)
        
        cosets.append(coset)
    
    return cosets


def multiplicative_group_mod_n(n: int) -> List[int]:
    """Возвращает элементы мультипликативной группы Z*_n."""
    return [i for i in range(1, n) if gcd(i, n) == 1]


def order_in_group(g: int, group: List[int], operation_mod: int) -> int:
    """Находит порядок элемента g в группе."""
    if g not in group:
        return 0

    power = g
    order = 1
    while power != 1:
        power = (power * g) % operation_mod
        order += 1
        if order > len(group):
            return 0
    return order


def mod_inverse(a: int, m: int) -> int:
    """Находит обратный элемент a^(-1) mod m."""
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1

        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y

    g, x, _ = extended_gcd(a % m, m)
    if g != 1:
        return None

    return (x % m + m) % m


def poly_divides(f: List[int], g: List[int], q: int) -> bool:
    """Проверяет, делится ли полином f на полином g над F_q."""
    f = f[:]
    g = g[:]
    while len(f) > 0 and f[0] == 0:
        f = f[1:]
    while len(g) > 0 and g[0] == 0:
        g = g[1:]
    
    if len(g) == 0 or len(g) > len(f):
        return False
    
    g_lead_inv = mod_inverse(g[0], q)
    if g_lead_inv is None:
        return False
    
    remainder = f[:]
    g_len = len(g)
    
    while len(remainder) >= g_len:
        if remainder[0] == 0:
            remainder = remainder[1:]
            continue
        
        factor = remainder[0]
        for i in range(g_len):
            if i < len(remainder):
                remainder[i] = (remainder[i] - factor * g[i] * g_lead_inv) % q
        
        while len(remainder) > 0 and remainder[0] == 0:
            remainder = remainder[1:]
    
    return len(remainder) == 0


def is_irreducible_poly(poly: List[int], q: int) -> bool:
    """Проверяет, является ли полином неприводимым над F_q."""
    poly = poly[:]
    while len(poly) > 0 and poly[0] == 0:
        poly = poly[1:]
    
    if len(poly) == 0:
        return False
    
    d = len(poly) - 1
    
    if d <= 1:
        return poly[-1] != 0
    
    max_divisor_deg = d // 2
    
    for deg in range(1, max_divisor_deg + 1):
        for divisor_coeffs in product(range(q), repeat=deg + 1):
            if divisor_coeffs[0] == 0:
                continue
            
            divisor = list(divisor_coeffs)
            
            if poly_divides(poly, divisor, q):
                return False
    
    return True


def poly_divide_with_quotient(f: List[int], g: List[int], q: int) -> Tuple[List[int], List[int]]:
    """Делит полином f на g, возвращает (частное, остаток)."""
    f = f[:]
    g = g[:]
    while len(f) > 0 and f[0] == 0:
        f = f[1:]
    while len(g) > 0 and g[0] == 0:
        g = g[1:]
    
    if len(g) > len(f):
        return ([0], f)
    
    g_lead_inv = mod_inverse(g[0], q)
    
    remainder = f[:]
    quotient = []
    g_len = len(g)
    f_len = len(f)
    
    while len(remainder) >= g_len:
        if remainder[0] == 0:
            quotient.append(0)
            remainder = remainder[1:]
            continue
        
        factor = (remainder[0] * g_lead_inv) % q
        quotient.append(factor)
        
        for i in range(g_len):
            if i < len(remainder):
                remainder[i] = (remainder[i] - factor * g[i]) % q
        
        while len(remainder) > 0 and remainder[0] == 0:
            remainder = remainder[1:]
    
    if not quotient:
        quotient = [0]
    
    while len(remainder) > 0 and remainder[0] == 0:
        remainder = remainder[1:]
    if not remainder:
        remainder = [0]
    
    return (quotient, remainder)


def poly_gcd(f: List[int], g: List[int], q: int) -> Tuple[List[int], List[int], List[int]]:
    """Находит gcd(f, g) и коэффициенты u, v такие что gcd = u*f + v*g (расширенный алгоритм Евклида)."""
    f = f[:]
    g = g[:]
    while len(f) > 0 and f[0] == 0:
        f = f[1:]
    while len(g) > 0 and g[0] == 0:
        g = g[1:]
    
    if len(f) == 0:
        if len(g) == 0:
            return ([0], [0], [0])
        g_lead_inv = mod_inverse(g[0], q)
        if g_lead_inv:
            g_normalized = [(coeff * g_lead_inv) % q for coeff in g]
            return (g_normalized, [0], [1])
        return (g, [0], [1])
    
    if len(g) == 0:
        f_lead_inv = mod_inverse(f[0], q)
        if f_lead_inv:
            f_normalized = [(coeff * f_lead_inv) % q for coeff in f]
            return (f_normalized, [1], [0])
        return (f, [1], [0])
    
    old_r, r = f[:], g[:]
    old_s, s = [1], [0]
    old_t, t = [0], [1]
    
    while len(r) > 0 and (len(r) > 1 or r[0] != 0):
        quotient, remainder = poly_divide_with_quotient(old_r, r, q)

        old_r, r = r, remainder

        def poly_mult(a: List[int], b: List[int], q: int) -> List[int]:
            """Умножает два полинома над F_q."""
            if len(a) == 0 or len(b) == 0:
                return [0]

            result = [0] * (len(a) + len(b) - 1)
            for i, coeff_a in enumerate(a):
                for j, coeff_b in enumerate(b):
                    result[i + j] = (result[i + j] + coeff_a * coeff_b) % q

            while len(result) > 0 and result[0] == 0:
                result = result[1:]
            if not result:
                return [0]
            return result
        
        def poly_sub(a: List[int], b: List[int], q: int) -> List[int]:
            """Вычитает полином b из полинома a над F_q."""
            max_len = max(len(a), len(b))
            result = [0] * max_len
            
            a_padded = [0] * (max_len - len(a)) + a if len(a) < max_len else a
            b_padded = [0] * (max_len - len(b)) + b if len(b) < max_len else b
            
            for i in range(max_len):
                result[i] = (a_padded[i] - b_padded[i]) % q
            
            while len(result) > 0 and result[0] == 0:
                result = result[1:]
            if not result:
                return [0]
            return result
        
        quot_s = poly_mult(quotient, s, q)
        quot_t = poly_mult(quotient, t, q)
        
        old_s, s = s, poly_sub(old_s, quot_s, q)
        old_t, t = t, poly_sub(old_t, quot_t, q)
    
    if len(old_r) > 0:
        r_lead_inv = mod_inverse(old_r[0], q)
        if r_lead_inv and old_r[0] != 0:
            gcd_normalized = [(coeff * r_lead_inv) % q for coeff in old_r]
            old_s_normalized = [(coeff * r_lead_inv) % q for coeff in old_s]
            old_t_normalized = [(coeff * r_lead_inv) % q for coeff in old_t]
            return (gcd_normalized, old_s_normalized, old_t_normalized)
    
    if len(old_r) == 0:
        return ([0], [0], [0])
    
    return (old_r, old_s, old_t)


def poly_mod_inverse(f: List[int], g: List[int], q: int) -> List[int]:
    """Находит f^(-1) mod g над F_q (оптимизированная версия)."""
    f = f[:]
    g = g[:]
    while len(f) > 0 and f[0] == 0:
        f = f[1:]
    while len(g) > 0 and g[0] == 0:
        g = g[1:]
    
    if len(f) == 0:
        return None
    
    gcd_poly, u, v = poly_gcd(f, g, q)
    
    gcd_normalized = gcd_poly[:]
    while len(gcd_normalized) > 0 and gcd_normalized[0] == 0:
        gcd_normalized = gcd_normalized[1:]
    
    if len(gcd_normalized) != 1 or gcd_normalized[0] != 1:
        return None
    
    _, remainder = poly_divide_with_quotient(u, g, q)
    
    while len(remainder) > 0 and remainder[0] == 0:
        remainder = remainder[1:]
    if not remainder:
        remainder = [0]
    
    return remainder



Найдите все подгруппы симметрической группы $S_m$.

Выведите их количество и одну случайную подгруппу. Для подгруппы с индексом $N \bmod (\text{число подгрупп})$ постройте левые и правые смежные классы, определите её индекс и проверьте, является ли она нормальной.

In [126]:
def subgroups_of_Sm(N: int) -> dict:
    m = 4 + (N % 5)
    
    S_m = generate_symmetric_group(m)
    all_subgroups = find_all_subgroups(S_m)
    
    random_subgroup = random.choice(all_subgroups)
    selected_subgroup = all_subgroups[N % len(all_subgroups)]

    left_cosets_list = left_cosets(S_m, selected_subgroup)
    right_cosets_list = right_cosets(S_m, selected_subgroup)

    left_cosets_frozen = {frozenset(coset) for coset in left_cosets_list}
    right_cosets_frozen = {frozenset(coset) for coset in right_cosets_list}
    is_normal = left_cosets_frozen == right_cosets_frozen
    
    return {
        'total_subgroups': len(all_subgroups),
        'random_subgroup_size': random_subgroup,
        'left_cosets': left_cosets_list,
        'right_cosets': right_cosets_list,
        'index': len(S_m) // len(selected_subgroup),
        'is_normal': is_normal
    }


In [ ]:
result = subgroups_of_Sm(N)
print(f"Всего подгрупп: {result['total_subgroups']}")
print(f"Размер случайной подгруппы: {len(result['random_subgroup_size'])}")
print(f"Индекс выбранной подгруппы: {result['index']}")
print(f"Является нормальной: {result['is_normal']}")


В группе $S_m$ возьмите элемент $g$ с индексом $N \bmod |S_m|$. Найдите порядки элементов $g^{n_1}$, $g^{n_2}$, $g^{n_3}$ и порядки циклических подгрупп, ими порождаемых.

In [70]:
def element_powers_in_Sm(N: int) -> dict:
    m = 4 + (N % 5)
    n1 = N % 6
    n2 = (N + 1) % 6
    n3 = (N + 2) % 6
    
    S_m = generate_symmetric_group(m)
    g = S_m[N % len(S_m)]
    
    g_n1 = g ** n1
    g_n2 = g ** n2
    g_n3 = g ** n3

    order_g_n1 = g_n1.order()
    order_g_n2 = g_n2.order()
    order_g_n3 = g_n3.order()
    
    # Порядки циклических подгрупп, порожденных этими элементами
    # Для циклической подгруппы порядок равен порядку порождающего элемента
    cyclic_subgroup_order_n1 = order_g_n1
    cyclic_subgroup_order_n2 = order_g_n2
    cyclic_subgroup_order_n3 = order_g_n3
    
    return {
        'ord(g^n1)': order_g_n1,
        'ord(g^n2)': order_g_n2,
        'ord(g^n3)': order_g_n3,
        'cyclic_subgroup_order(g^n1)': cyclic_subgroup_order_n1,
        'cyclic_subgroup_order(g^n2)': cyclic_subgroup_order_n2,
        'cyclic_subgroup_order(g^n3)': cyclic_subgroup_order_n3
    }


In [71]:
result = element_powers_in_Sm(N)
print(f"ord(g^n1): {result['ord(g^n1)']}")
print(f"ord(g^n2): {result['ord(g^n2)']}")
print(f"ord(g^n3): {result['ord(g^n3)']}")
print(f"cyclic_subgroup_order(g^n1): {result['cyclic_subgroup_order(g^n1)']}")
print(f"cyclic_subgroup_order(g^n2): {result['cyclic_subgroup_order(g^n2)']}")
print(f"cyclic_subgroup_order(g^n3): {result['cyclic_subgroup_order(g^n3)']}")

ord(g^n1): 4
ord(g^n2): 1
ord(g^n3): 4
cyclic_subgroup_order(g^n1): 4
cyclic_subgroup_order(g^n2): 1
cyclic_subgroup_order(g^n3): 4


В группе $S_m$ найдите все решения уравнения $\sigma^n = (1\ 2\ 3\ \ldots\ m-1)$.

Выведите количество решений и три случайных решения. Опишите, что у них общего.

In [72]:
def solve_sigma_power_eq(N: int) -> dict:
    m = 4 + (N % 5)
    n = 2 + (N % 10)
    
    S_m = generate_symmetric_group(m)
    target = Permutation(list(range(2, m + 1)) + [1])
    
    solutions = []
    for sigma in S_m:
        if (sigma ** n) == target:
            solutions.append(sigma)
    
    return {
        'solutions': solutions
    }


In [73]:
result = solve_sigma_power_eq(N)
print(f"Количество решений: {len(result['solutions'])}")
if result['solutions']:
    print(f"Первые 3 решения:")
    for i, sol in enumerate(result['solutions'][:3]):
        print(f"  {i+1}. {sol}")


Количество решений: 1
Первые 3 решения:
  1. (1 4 7 2 5 8 3 6)


В циклической группе порядка $m$ найдите:

- все элементы $g$, такие что $g^k = e$;
- все элементы порядка $k$.

In [76]:
def elements_of_order_k_in_cyclic_group(N: int) -> dict:
    m = 4 + (N % 5)
    k = 1 + (N % 7)

    elements_g_k_e = []
    d = gcd(m, k)
    step = m // d if d > 0 else m
    for g in range(0, m, step):
        elements_g_k_e.append(g)

    elements_order_k = []
    if m % k == 0:
        divisor = m // k
        for g in range(m):
            if gcd(m, g) == divisor:
                elements_order_k.append(g)
    
    return {
        'elements_g_such_that_g^k_e': elements_g_k_e,
        'elements_of_order_k': elements_order_k
    }


In [77]:
result = elements_of_order_k_in_cyclic_group(N)
print(f"Элементы g такие, что g^k = e: {result['elements_g_such_that_g^k_e']}")
print(f"Элементы порядка k: {result['elements_of_order_k']}")


Элементы g такие, что g^k = e: [0]
Элементы порядка k: []


Найдите все подгруппы мультипликативной группы $\mathbb{Z}_m^*$.


In [78]:
def subgroups_of_Zm_star(N: int) -> list:
    m = 4 + (N % 5)

    Z_m_star = multiplicative_group_mod_n(m)
    
    if not Z_m_star:
        return []
    
    subgroups = [[1]]

    for r in range(2, min(len(Z_m_star) + 1, 10)):
        for subset in combinations(Z_m_star, r):
            subgroup = is_subgroup(list(subset), m)
            if subgroup and subgroup not in subgroups:
                subgroups.append(subgroup)
    
    if Z_m_star not in subgroups:
        subgroups.append(Z_m_star)
    
    return subgroups


def is_subgroup(elements: List[int], mod: int) -> List[int]:
    """Проверяет, является ли множество элементов подгруппой."""
    if 1 not in elements:
        return None

    for a in elements:
        for b in elements:
            product = (a * b) % mod
            if product not in elements:
                return None

    for a in elements:
        found_inverse = False
        for b in elements:
            if (a * b) % mod == 1:
                found_inverse = True
                break
        if not found_inverse:
            return None

    return sorted(elements)


In [79]:
result = subgroups_of_Zm_star(N)
print(f"Количество подгрупп: {len(result)}")
print(f"Подгруппы:")
for i, subgroup in enumerate(result):
    print(f"  {i+1}. {subgroup}")


Количество подгрупп: 5
Подгруппы:
  1. [1]
  2. [1, 3]
  3. [1, 5]
  4. [1, 7]
  5. [1, 3, 5, 7]


Пусть $s \in \mathbb{Z}_p^*$. Найдите порядок элемента $s^r$ в $\mathbb{Z}_p^*$.

In [80]:
def order_of_sr(N: int) -> int:
    lookup = {
        0: {'p': 29, 's': 5, 'r': 59},
        1: {'p': 31, 's': 4, 'r': 60},
        2: {'p': 37, 's': 3, 'r': 38},
        3: {'p': 23, 's': 17, 'r': 45},
        4: {'p': 19, 's': 15, 'r': 44}
    }
    params = lookup[N % 5]
    p, s, r = params['p'], params['s'], params['r']

    Z_p_star = multiplicative_group_mod_n(p)
    s_r = pow(s, r, p)
    order = order_in_group(s_r, Z_p_star, p)

    return order


In [81]:
result = order_of_sr(N)
print(f"Порядок элемента s^r: {result}")


Порядок элемента s^r: 9


Найдите порядок элемента $t$ в группе $\mathbb{Z}_p^*$. Является ли $t$ образующим (примитивным корнем)?

In [82]:
def order_and_primitivity_of_t(N: int) -> dict:
    lookup = {
        0: {'p': 29, 't': 9},
        1: {'p': 31, 't': 8},
        2: {'p': 37, 't': 7},
        3: {'p': 23, 't': 12},
        4: {'p': 19, 't': 14}
    }
    params = lookup[N % 5]
    p, t = params['p'], params['t']
    
    Z_p_star = multiplicative_group_mod_n(p)
    order_t = order_in_group(t, Z_p_star, p)
    is_primitive = (order_t == len(Z_p_star))
    
    return {
        't': t,
        'ord(t)': order_t,
        'is_primitive_root': is_primitive
    }


In [83]:
result = order_and_primitivity_of_t(N)
print(f"t = {result['t']}")
print(f"ord(t) = {result['ord(t)']}")
print(f"Является примитивным корнем: {result['is_primitive_root']}")


t = 14
ord(t) = 18
Является примитивным корнем: True


Найдите все образующие (примитивные корни) циклической группы $\mathbb{Z}_m^*$.

In [84]:
def generators_of_Zm_star(N: int) -> list:
    m = 4 + (N % 5)
    
    Z_m_star = multiplicative_group_mod_n(m)

    group_order = len(Z_m_star)
    generators = []

    for g in Z_m_star:
        order_g = order_in_group(g, Z_m_star, m)
        if order_g == group_order:
            generators.append(g)
    
    return sorted(generators)


In [85]:
result = generators_of_Zm_star(N)
print(f"Количество образующих: {len(result)}")
print(f"Образующие: {result}")


Количество образующих: 0
Образующие: []


В аддитивной группе $\mathbb{Z}_m$ найдите циклическую подгруппу, порождённую элементом $t \bmod m$. Определите её порядок и все порождающие элементы. Опишите связь с исходным $t$.

In [86]:
def cyclic_subgroup_in_Zm_additive(N: int) -> dict:
    m = 4 + (N % 5)
    lookup = {
        0: {'t': 9},
        1: {'t': 8},
        2: {'t': 7},
        3: {'t': 12},
        4: {'t': 14}
    }
    t = lookup[N % 5]['t']
    
    subgroup = []
    current = 0
    visited = set()
    
    while current not in visited:
        visited.add(current)
        subgroup.append(current)
        current = (current + t) % m
    
    order = len(subgroup)
    
    d = gcd(m, t)
    subgroup_order = m // d
    generators = []
    for k in range(subgroup_order):
        if gcd(k, subgroup_order) == 1:
            generators.append((k * t) % m)
    
    return {
        'subgroup': sorted(subgroup),
        'ord': order,
        'generators': sorted(generators)
    }


In [87]:
result = cyclic_subgroup_in_Zm_additive(N)
print(f"Подгруппа: {result['subgroup']}")
print(f"Порядок: {result['ord']}")
print(f"Порождающие элементы: {result['generators']}")

Подгруппа: [0, 2, 4, 6]
Порядок: 4
Порождающие элементы: [2, 6]


В мультипликативной группе $\mathbb{Z}_m^*$ найдите циклическую подгруппу, порождённую элементом $t \bmod m$. Определите, какой циклической подгруппе в симметрической группе $S_d$ (где $d$ — порядок подгруппы) она изоморфна.


In [88]:
def isomorphism_of_cyclic_subgroup_Zm_star(N: int) -> dict:
    m = 4 + (N % 5)
    lookup = {
        0: {'t': 9},
        1: {'t': 8},
        2: {'t': 7},
        3: {'t': 12},
        4: {'t': 14}
    }
    t = lookup[N % 5]['t']
    
    Z_m_star = multiplicative_group_mod_n(m)
    
    subgroup = []
    current = 1
    power = 0

    while True:
        if current in subgroup:
            break
        subgroup.append(current)
        power += 1
        current = (current * t) % m
    
    d = len(subgroup)
    
    return {
        'subgroup': sorted(subgroup),
        'isomorphic_to': list(range(1, d + 1)),
    }


In [89]:
result = isomorphism_of_cyclic_subgroup_Zm_star(N)
print(f"Подгруппа: {result['subgroup']}")
print(f"Изоморфна циклической подгруппе в S_d, где d = {len(result['subgroup'])}: {result['isomorphic_to']}")


Подгруппа: [0, 1, 4, 6]
Изоморфна циклической подгруппе в S_d, где d = 4: [1, 2, 3, 4]


Найдите все корни полиномов:

- $f_1(x) = x^9 + \sum_{i=0}^{8} a_i x^i \in \mathbb{F}_4[x]$;
- $f_2(x) = \sum_{i=0}^{6} b_i x^i \in \mathbb{F}_7[x]$.

In [159]:
def find_all_poly_roots(N: int) -> dict:
    """
    Находит все корни полиномов над конечными полями.
    """
    f1 = [1] + [(i + N) % 4 for i in range(9)]
    f2 = [(j + N) % 7 for j in range(7)]

    def poly_eval(poly, x, q):
        """Вычисляет значение полинома в точке x над F_q."""
        result = 0
        for i, coeff in enumerate(poly):
            power = len(poly) - 1 - i
            result = (result + coeff * pow(x, power, q)) % q
        return result
    
    def find_roots(poly, q):
        """Находит все корни полинома над F_q."""
        roots = []
        for x in range(q):
            if poly_eval(poly, x, q) == 0:
                roots.append(x)
        return roots
    
    roots_f1 = find_roots(f1, 4)
    roots_f2 = find_roots(f2, 7)
    
    return {
        'f1_coefficients': f1,
        'f1_roots_F4': roots_f1,
        'f2_coefficients': f2,
        'f2_roots_F7': roots_f2
    }


In [160]:
result = find_all_poly_roots(N)
print(f"f1(x) коэффициенты: {result['f1_coefficients']}")
print(f"Корни f1(x) в F_4: {result['f1_roots_F4']}")
print(f"f2(x) коэффициенты: {result['f2_coefficients']}")
print(f"Корни f2(x) в F_7: {result['f2_roots_F7']}")


f1(x) коэффициенты: [1, 1, 2, 3, 0, 1, 2, 3, 0, 1]
Корни f1(x) в F_4: [3]
f2(x) коэффициенты: [2, 3, 4, 5, 6, 0, 1]
Корни f2(x) в F_7: [1, 4]


Исследуйте полиномы на приводимость. Приводимые полиномы разложите на неприводимые множители:

- $f_3(x) = x^5 + \sum_{i=0}^{4} c_i x^i \in \mathbb{F}_5[x]$;

- $f_4(x) = x^4 + \sum_{i=0}^{3} d_i x^i \in \mathbb{F}_9[x]$.

In [161]:
def investigate_poly_reducibility(N: int) -> dict:
    """
    Исследует полиномы на приводимость и разлагает приводимые на неприводимые множители.
    """
    f3 = [1] + [(k + N) % 5 for k in range(5)]
    f4 = [1] + [(l + N) % 9 for l in range(4)]

    def factorize_poly(poly, q):
        """Разлагает полином на неприводимые множители."""
        factors = []
        remaining = poly[:]
    
        while len(remaining) > 0 and remaining[0] == 0:
            remaining = remaining[1:]
        
        if len(remaining) == 0:
            return [[0]]
        
        d = len(remaining) - 1
        if d <= 1:
            return [remaining]
    
        for deg in range(1, d // 2 + 1):
            for divisor_coeffs in product(range(q), repeat=deg + 1):
                if divisor_coeffs[0] == 0:
                    continue
                divisor = list(divisor_coeffs)
                
                if poly_divides(remaining, divisor, q):
                    factors.append(divisor)
                    quotient, _ = poly_divide_with_quotient(remaining, divisor, q)
                    if quotient is not None:
                        remaining = quotient
                        d = len(remaining) - 1
                        if d <= 1:
                            factors.append(remaining)
                            return factors
                        break
        
        if not factors:
            return [remaining]
        
        if len(remaining) > 1:
            factors.append(remaining)
        
        return factors
    
    is_irred_f3 = is_irreducible_poly(f3, 5)
    factors_f3 = factorize_poly(f3, 5) if not is_irred_f3 else [f3]
    
    is_irred_f4 = is_irreducible_poly(f4, 9)
    factors_f4 = factorize_poly(f4, 9) if not is_irred_f4 else [f4]
    
    return {
        'f3_coefficients': f3,
        'f3_is_irreducible': is_irred_f3,
        'f3_factors': factors_f3,
        'f4_coefficients': f4,
        'f4_is_irreducible': is_irred_f4,
        'f4_factors': factors_f4
    }


In [162]:
result = investigate_poly_reducibility(N)
print(f"f3(x) коэффициенты: {result['f3_coefficients']}")
print(f"f3(x) неприводим: {result['f3_is_irreducible']}")
print(f"Множители f3(x): {result['f3_factors']}")
print(f"f4(x) коэффициенты: {result['f4_coefficients']}")
print(f"f4(x) неприводим: {result['f4_is_irreducible']}")
print(f"Множители f4(x): {result['f4_factors']}")


f3(x) коэффициенты: [1, 4, 0, 1, 2, 3]
f3(x) неприводим: False
Множители f3(x): [[1, 1], [1, 4, 2], [1, 4, 4]]
f4(x) коэффициенты: [1, 0, 1, 2, 3]
f4(x) неприводим: False
Множители f4(x): [[1, 6], [1, 5, 2], [1, 7]]


Пусть
$$f(x) = \sum_{i=0}^{7} r_i x^i, \quad g(x) = \sum_{i=0}^{3} s_i x^i$$
— полиномы над полем $\mathbb{F}_{11}$. Найдите $\gcd(f, g)$ и его линейное представление:
$$\gcd(f, g) = u(x)f(x) + v(x)g(x).$$

In [163]:
def find_poly_gcd_with_representation(N: int) -> dict:
    """
    Находит gcd(f, g) и его линейное представление gcd = u*f + v*g над F_11.
    """
    f = [(m + N) % 11 for m in range(8)]
    g = [(t + N) % 11 for t in range(4)]

    gcd_poly, u, v = poly_gcd(f, g, 11)
    
    return {
        'f_coefficients': f,
        'g_coefficients': g,
        'gcd': gcd_poly,
        'u_coefficients': u,
        'v_coefficients': v,
    }


In [164]:
result = find_poly_gcd_with_representation(N)
print(f"f(x) коэффициенты: {result['f_coefficients']}")
print(f"g(x) коэффициенты: {result['g_coefficients']}")
print(f"gcd(f, g): {result['gcd']}")
print(f"u(x) коэффициенты: {result['u_coefficients']}")
print(f"v(x) коэффициенты: {result['v_coefficients']}")



f(x) коэффициенты: [10, 0, 1, 2, 3, 4, 5, 6]
g(x) коэффициенты: [10, 0, 1, 2]
gcd(f, g): [1]
u(x) коэффициенты: [2, 7]
v(x) коэффициенты: [9, 10, 7]


Пусть
$$f(x) = s_2 x^2 + s_1 x + s_0, \quad g(x) = x^8 + x^4 + x^3 + 6x + 2$$
— полиномы над полем $\mathbb{F}_{13}$. Найдите обратный элемент $f^{-1} \bmod g$, то есть такой полином $h$, что
$$f(x)h(x) \equiv 1 \pmod{g(x)}.$$

In [165]:
def find_poly_mod_inverse(N: int) -> dict:
    """
    Находит обратный элемент f^(-1) mod g над F_13.
    """
    f = [(t + N) % 13 for t in range(4)]
    g = [1, 0, 0, 0, 1, 1, 0, 6, 2]

    inv_f = poly_mod_inverse(f, g, 13)
    
    return {
        'f_coefficients': f,
        'g_coefficients': g,
        'f_inverse_mod_g': inv_f
    }


In [167]:
result = find_poly_mod_inverse(N)
print(f"f(x) коэффициенты: {result['f_coefficients']}")
print(f"g(x) коэффициенты: {result['g_coefficients']}")
print(f"f^(-1) mod g: {result['f_inverse_mod_g']}")

f(x) коэффициенты: [1, 2, 3, 4]
g(x) коэффициенты: [1, 0, 0, 0, 1, 1, 0, 6, 2]
f^(-1) mod g: [5, 4, 2, 7, 8, 0, 3]


Реализуйте алгоритм генерации всех неприводимых полиномов степени $d$ над простым конечным полем $\mathbb{F}_q$, где $q$ — простое. Протестируйте его для $q = 2, 3, 5$ и $d = 2, 3, 4$.


In [94]:
def generate_irreducible_polynomials(q: int, d: int) -> list:
    """
    Возвращает список всех неприводимых полиномов степени d над F_q.
    Полином представляется как список коэффициентов [a_d, a_{d-1}, ..., a_0],
    где полином = a_d*x^d + a_{d-1}*x^{d-1} + ... + a_0.
    """
    polynomials = []
    
    for coeffs in product(range(q), repeat=d + 1):
        if coeffs[0] != 0:
            poly = list(coeffs)
            if is_irreducible_poly(poly, q):
                polynomials.append(poly)
    
    return polynomials


In [95]:
for q in [2, 3, 5]:
    for d in [2, 3, 4]:
        result = generate_irreducible_polynomials(q, d)
        print(f"q={q}, d={d}: найдено {len(result)} неприводимых полиномов")
        if result:
            print(f"  Примеры: {result[:3]}")


q=2, d=2: найдено 1 неприводимых полиномов
  Примеры: [[1, 1, 1]]
q=2, d=3: найдено 2 неприводимых полиномов
  Примеры: [[1, 0, 1, 1], [1, 1, 0, 1]]
q=2, d=4: найдено 3 неприводимых полиномов
  Примеры: [[1, 0, 0, 1, 1], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]
q=3, d=2: найдено 6 неприводимых полиномов
  Примеры: [[1, 0, 1], [1, 1, 2], [1, 2, 2]]
q=3, d=3: найдено 16 неприводимых полиномов
  Примеры: [[1, 0, 2, 1], [1, 0, 2, 2], [1, 1, 0, 2]]
q=3, d=4: найдено 36 неприводимых полиномов
  Примеры: [[1, 0, 0, 1, 2], [1, 0, 0, 2, 2], [1, 0, 1, 0, 2]]
q=5, d=2: найдено 40 неприводимых полиномов
  Примеры: [[1, 0, 2], [1, 0, 3], [1, 1, 1]]
q=5, d=3: найдено 160 неприводимых полиномов
  Примеры: [[1, 0, 1, 1], [1, 0, 1, 4], [1, 0, 2, 1]]
q=5, d=4: найдено 600 неприводимых полиномов
  Примеры: [[1, 0, 0, 0, 2], [1, 0, 0, 0, 3], [1, 0, 0, 1, 4]]
